In [2]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
from tqdm import tqdm
import re
import sys

In [3]:
df = pd.read_excel(r'C:/Users/smhrd/food_info_.xlsx')
df

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,식품군,제조사명,지역명,중량,열량(kcal),탄수화물(g),단백질(g),지방(g),당류(g),나트륨(mg),콜레스트롤(mg),포화지방산(g),트랜스지방(g)
0,1,꿩불고기,https://www.youtube.com/embed/rl5xaIdHjPY,북,메,NaN,NaN,충주,500.0,368.80,39.70,33.50,8.50,16.90,1264.31,106.18,1.90,0.10
1,2,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,NaN,NaN,전국(대표),400.0,595.61,44.90,45.90,25.80,21.20,1535.83,193.40,6.00,0.20
2,3,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,NaN,NaN,춘천,300.0,558.47,23.10,45.50,31.60,8.50,1016.94,217.67,11.30,0.30
3,4,닭꼬치,https://www.youtube.com/embed/1xrcjy6eX7M,한,메,NaN,NaN,전국(대표),70.0,176.72,13.35,11.56,8.57,3.15,286.91,31.44,2.32,0.04
4,5,더덕구이,https://www.youtube.com/embed/ir4Z2IyFIvM,한,반,NaN,NaN,전국(대표),100.0,184.00,31.10,3.10,5.20,11.60,743.37,0.12,0.80,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90603,90604,"코코아, 가루",NaN,NaN,NaN,차류,NaN,전국(대표),100.0,268.00,43.63,22.85,20.06,0.49,24.00,0.00,12.11,0.01
90604,90605,"홍삼차, 액상",NaN,NaN,NaN,차류,NaN,전국(대표),100.0,40.00,9.27,0.28,0.24,6.32,4.00,0.00,0.00,0.00
90605,90606,"홍차, 복숭아, 가루",NaN,NaN,NaN,차류,NaN,전국(대표),100.0,397.00,98.12,0.32,0.38,42.31,41.00,0.00,0.00,0.00
90606,90607,"마요네즈, 저칼로리",NaN,NaN,NaN,조미료류,NaN,전국(대표),100.0,341.00,12.35,0.89,31.97,3.56,600.00,23.72,3.96,0.22


In [87]:
# 필요없는 부분 -> Food_group, Food_mf 등등 삭제
# 쓸 부분 - seq, name, recipe, weigth, calor, crb, protein, fat 씁니다.
# 컬럼 제거
df.drop(
    ['식품군','제조사명','지역명','당류(g)','나트륨(mg)','콜레스트롤(mg)'
    ,'포화지방산(g)','트랜스지방(g)'],
    axis=1,
    errors='ignore',
    inplace=True
)

In [88]:
# Food_name 전처리하기
df['음식명'] = df['음식명'].apply(lambda x: re.sub('\s(?![a-z])|_|\((.*?)\)|(.*?)\?','', str(x)))
df.head()

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal),탄수화물(g),단백질(g),지방(g)
0,1,꿩불고기,https://www.youtube.com/embed/rl5xaIdHjPY,북,메,500.0,368.80,39.70,33.50,8.50
1,2,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,400.0,595.61,44.90,45.90,25.80
2,3,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,300.0,558.47,23.10,45.50,31.60
3,4,닭꼬치,https://www.youtube.com/embed/1xrcjy6eX7M,한,메,70.0,176.72,13.35,11.56,8.57
4,5,더덕구이,https://www.youtube.com/embed/ir4Z2IyFIvM,한,반,100.0,184.00,31.10,3.10,5.20


In [102]:
# 개인 정보에 기반한 활동 대사량, 기초대사량 구하기
# Physical Activity Level (PAL)

# *args로 하면 가변인자를 사용할 수 있다고 하네요!!
def user_PAL(*args):
    
    weight, height, age, gender, activity, purpose = args
    # 기초대사량 -> 키, 몸무게, 나이로 구성된 기본식
    base = 10 * weight + 6.25 * height - 5 * age

    # 성별에 따른 계산
    if gender == 'm':
        man_base = base + 5
        crb, protein, fat = 0.5, 0.15, 0.35
        user = {'cal': man_base, 'crb': crb, 'protein': protein, 'fat': fat}
    else:
        woman_base = base - 161
        crb, protein, fat = 0.5, 0.15, 0.35
        user = {'cal': woman_base, 'crb': crb, 'protein': protein, 'fat': fat}

    # Eat-Fit 목적에 따른 계산
    if purpose == 'plus':
        user['cal'] *= 1.2
    elif purpose == 'lose':
        user['cal'] *= 0.8

    # 활동량에 따른 계산
    activity_levels = {1: 1.2, 2: 1.375, 3: 1.55, 4: 1.725, 5: 1.9}
    user['cal'] *= activity_levels[activity]
    user['cal'] = int(user['cal'])
    
    # 단위를 맞춰주기 위해 계산 결과를 조정
    user['crb'] = int(round(user['cal'] * crb / 4, 1))
    user['protein'] = int(round(user['cal'] * protein / 4,1))
    user['fat'] = int(round(user['cal'] * fat / 9,1))

    return user

# 내가 가진 영양정보에서 오늘 먹은 식단 제외하고 남은 영양 성분 구하는 공식
# 여기서 x는 먹은 음식의 food_seq + 1입니다.
def remaining_nut(dic, df, x):
    # 필요한 변수 정의
    user_cal, user_crb, user_protein, user_fat = dic['cal'], dic['crb'], dic['protein'], dic['fat']
    food_cal, food_crb, food_protein, food_fat = df.loc[x,'열량(kcal)'], df.loc[x,'탄수화물(g)'], df.loc[x,'단백질(g)'], df.loc[x,'지방(g)']

    # 각각의 영양 성분에서 먹은 식단의 섭취량을 빼서 남은 영양 성분 구하기
    remain_cal, remain_crb, remain_protein, remain_fat = user_cal - food_cal, user_crb - food_crb, user_protein - food_protein, user_fat - food_fat

    # 딕셔너리 형태로 저장.
    return {'cal': round(remain_cal, 3), 'crb': round(remain_crb, 3), 'protein': round(remain_protein, 3), 'fat': round(remain_fat, 3)}
  

In [103]:
user_status =[72,176,25,'m',3,'maintain']
user_status

[72, 176, 25, 'm', 3, 'maintain']

In [104]:
user = user_PAL(*user_status)
user

{'cal': 2635, 'crb': 329, 'protein': 98, 'fat': 102}

In [105]:
# def kor_crb(df,user)
# 전체 df에서 한식, 일식만 가져옴
kor_jap_df = df[df['한중일양'].isin(['한', '일'])]
# 한,일식 df에서 밥,면 종류만 가져옴
rice_noodle_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['밥','면'])]
rice_noodle_df

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal),탄수화물(g),단백질(g),지방(g)
20,21,굴국밥,https://www.youtube.com/embed/0T2EXWPgYS4,한,밥,650.0,683.00,88.40,42.40,17.80
73,74,김치라면,https://www.youtube.com/embed/1H5qj5Yl4kQ,한,면,650.0,552.00,86.20,13.80,16.90
74,75,김치우동,https://www.youtube.com/embed/toYOkxNIUn8,한,면,800.0,500.18,99.14,15.55,4.60
75,76,냉김치말이국수,https://www.youtube.com/embed/s28sdWURHfs,한,면,600.0,302.53,52.00,12.90,4.80
76,77,닭칼국수,https://www.youtube.com/embed/x-IeGK4y0wU,한,면,900.0,663.00,94.70,42.20,12.80
...,...,...,...,...,...,...,...,...,...,...
7241,7242,전복죽,https://www.youtube.com/embed/XIyrc8JMgv4,한,밥,400.0,317.19,38.49,7.27,14.90
7242,7243,채소죽,https://www.youtube.com/embed/mabqkgdjV8I,한,밥,400.0,173.76,30.47,6.75,2.76
7243,7244,팥죽,https://www.youtube.com/embed/AOF_2SjfwMs,한,밥,400.0,324.17,66.63,12.69,0.76
7244,7245,호박죽,https://www.youtube.com/embed/PzL4Y3LYBpU,한,밥,400.0,197.52,26.30,7.70,6.84


In [106]:
print(rice_noodle_df.iloc[0]['탄수화물(g)'])
print(" ")
print(rice_noodle_df.iloc[0])

88.4
 
FOOD_SEQ                                           21
음식명                                               굴국밥
레시피         https://www.youtube.com/embed/0T2EXWPgYS4
한중일양                                                한
밥국반메면                                               밥
중량                                              650.0
열량(kcal)                                        683.0
탄수화물(g)                                          88.4
단백질(g)                                           42.4
지방(g)                                            17.8
Name: 20, dtype: object


In [107]:
need_crb=user['crb']/3
need_cal = user['cal']/3
need_protein=user['protein']/3
need_fat = user['fat']/3

In [128]:
food_crb_lst = []
mem_remain_crb_lst = []
for i in range(len(rice_noodle_df)):
    food_crb=rice_noodle_df.iloc[i]['탄수화물(g)']
    food_cal=rice_noodle_df.iloc[i]['열량(kcal)']
    food_protein=rice_noodle_df.iloc[i]['단백질(g)']
    food_fat=rice_noodle_df.iloc[i]['지방(g)']
    
    remain_user_crb = need_crb
    remain_user_cal = need_cal
    remain_user_protein = need_protein
    remain_user_fat = need_fat
    
    if (need_crb*0.60) <= food_crb <= (need_crb*0.85):
        food_crb_lst.append(rice_noodle_df.iloc[i])
        remain_crb = int(round(remain_user_crb - food_crb,1))
        remain_cal = int(round(remain_user_cal - food_cal,1))
        remain_protein = int(round(remain_user_protein - food_protein,1))
        remain_fat = int(round(remain_user_fat - food_fat,1))
        
        mem_remain_crb_lst.append({'FOOD_RICE_SEQ':rice_noodle_df.iloc[i]['FOOD_SEQ'] ,'remain_crb':remain_crb,
                               'remain_cal':remain_cal, 'remain_protein':remain_protein, 'remain_fat':remain_fat})
#print(user['crb']) # 이렇게 하면 마지막 나온 친구에서 crb가 빠짐
food_crb_df = pd.DataFrame(food_crb_lst)
mem_remain_crb_df = pd.DataFrame(mem_remain_crb_lst)

print(len(food_crb_df))
print(len(mem_remain_crb_df))
# cut_crb_df = pd.merge(food_crb_df,mem_remain_crb_df)
# cut_crb_df

# 여기서 return할때 remain_머시기들 중에 3개가 음수면 그거 추천해주는 걸로 가고


58
58


In [130]:
mem_remain_crb_df

,FOOD_RICE_SEQ,remain_crb,remain_cal,remain_protein,remain_fat
0,21,21,195,-9,16
1,74,23,326,18,17
2,84,40,447,17,23
3,88,35,456,19,26
4,92,17,189,11,7
5,93,29,283,15,11
6,116,30,481,15,32
7,117,35,-33,-33,-4
8,119,19,331,16,20
9,122,27,486,18,33


### 국을 찾아봅시다

In [115]:
# def kor_crb(df,user)
# 전체 df에서 한식, 일식만 가져옴
# kor_jap_df = df[df['한중일양'].isin(['한', '일'])]
# 얘는 이미 고정
# 한,일식 df에서 밥,면 종류만 가져옴
soup_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['국'])]
soup_df

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal),탄수화물(g),단백질(g),지방(g)
18,19,게국지,https://www.youtube.com/embed/Q98Wie1p7tc,한,국,1100.0,415.25,31.70,44.90,12.10
19,20,곰치국,https://www.youtube.com/embed/2yt5fIVJquo,한,국,1000.0,419.29,23.90,54.30,11.80
21,22,김치국,https://www.youtube.com/embed/qEjH1-fVNbg,한,국,450.0,89.00,10.50,5.40,2.80
22,23,떡만둣국,https://www.youtube.com/embed/At-u3lwsFtk,한,국,700.0,624.92,110.71,20.06,11.32
23,24,만둣국,https://www.youtube.com/embed/shWm_irzdpk,한,국,700.0,434.05,47.83,21.43,17.45
...,...,...,...,...,...,...,...,...,...,...
7283,7284,콩나물된장찌개,https://www.youtube.com/embed/o1eDvlKeEK4,한,국,300.0,146.27,18.94,7.69,4.42
7284,7285,콩비지찌개,https://www.youtube.com/embed/hz2QcmE5WuU,한,국,400.0,275.73,14.27,26.95,12.32
7285,7286,해물매운탕,https://www.youtube.com/embed/vZTlVM7iadY,한,국,600.0,211.37,15.08,29.07,3.86
7286,7287,햄김치찌개,https://www.youtube.com/embed/swO5pOM3Rg0,한,국,300.0,93.41,9.52,4.42,4.18


In [137]:
food_soup_lst = []
mem_remain_cal_lst = []
for i in range(len(soup_df)):
    food_crb=soup_df.iloc[i]['탄수화물(g)']
    food_cal=soup_df.iloc[i]['열량(kcal)']
    food_protein=soup_df.iloc[i]['단백질(g)']
    food_fat=soup_df.iloc[i]['지방(g)']
    for j in range(len(mem_remain_df)):
        remain_user_crb = mem_remain_crb_df.iloc[j]['remain_crb']
        remain_user_cal = mem_remain_crb_df.iloc[j]['remain_cal']
        remain_user_protein = mem_remain_crb_df.iloc[j]['remain_protein']
        remain_user_fat = mem_remain_crb_df.iloc[j]['remain_fat']
    
        if (need_cal*0.1) <= food_cal <= (need_cal*0.2):
            food_soup_lst.append(soup_df.iloc[i])
            remain_crb = int(round(remain_user_crb - food_crb,1))
            remain_cal = int(round(remain_user_cal - food_cal,1))
            remain_protein = int(round(remain_user_protein - food_protein,1))
            remain_fat = int(round(remain_user_fat - food_fat,1))
            mem_remain_cal_lst.append({'FOOD_SOUP_SEQ':soup_df.iloc[i]['FOOD_SEQ'],'FOOD_RICE_SEQ':mem_remain_crb_df.iloc[j]['FOOD_RICE_SEQ'] ,'remain_crb':remain_crb,
                                       'remain_cal':remain_cal, 'remain_protein':remain_protein, 'remain_fat':remain_fat})

#print(user['crb']) # 이렇게 하면 마지막 나온 친구에서 crb가 빠짐
food_soup_df = pd.DataFrame(food_soup_lst)
mem_remain_cal_df = pd.DataFrame(mem_remain_cal_lst)
print(len(food_soup_lst))
print(len(food_soup_df))
print(len(mem_remain_cal_df))
# cut_soup_df = pd.merge(food_soup_df,mem_remain_soup_df)
# cut_soup_df

2900
2900
2900


In [138]:
mem_remain_cal_df

,FOOD_SOUP_SEQ,FOOD_RICE_SEQ,remain_crb,remain_cal,remain_protein,remain_fat
0,22,21,10,106,-14,13
1,22,74,12,237,12,14
2,22,84,29,358,11,20
3,22,88,24,367,13,23
4,22,92,6,100,5,4
...,...,...,...,...,...,...
2895,7288,7019,15,361,12,26
2896,7288,7020,3,367,19,29
2897,7288,7021,12,414,20,30
2898,7288,7241,12,144,-3,11


In [139]:
# cal 마이너스인거 삭제하기
i = 0
while i < len(mem_remain_cal_df):
    if mem_remain_cal_df.iloc[i]['remain_cal']<0:
        mem_remain_cal_df = mem_remain_cal_df.drop(i)
    else:
        i += 1
print(len(mem_remain_cal_df))

2800


- 3번째 메인 반찬 찾기.

In [142]:
main_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['메'])]
main_df

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal),탄수화물(g),단백질(g),지방(g)
1,2,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,400.0,595.61,44.90,45.90,25.80
2,3,닭갈비,https://www.youtube.com/embed/HI8DlDs1gko,한,메,300.0,558.47,23.10,45.50,31.60
3,4,닭꼬치,https://www.youtube.com/embed/1xrcjy6eX7M,한,메,70.0,176.72,13.35,11.56,8.57
5,6,도미구이,https://www.youtube.com/embed/Q0W5ag8ZbzU,한,메,250.0,397.00,2.10,61.10,16.50
6,7,돼지갈비,https://www.youtube.com/embed/hxUqpXTIJ-8,한,메,100.0,240.32,8.10,19.50,14.40
...,...,...,...,...,...,...,...,...,...,...
7321,7322,새우튀김,https://www.youtube.com/embed/EPLbReYKHC8,한,메,150.0,430.34,28.44,25.52,23.84
7322,7323,소시지튀김,https://www.youtube.com/embed/3zrCyzg0xdk,한,메,200.0,707.98,8.01,28.13,62.60
7323,7324,오징어튀김,https://www.youtube.com/embed/K7KCOvEKPbg,한,메,150.0,505.71,33.65,18.56,32.99
7324,7325,채소튀김,https://www.youtube.com/embed/1uvcqAUjH8A,한,메,100.0,404.63,36.83,3.25,27.14


In [160]:
food_main_lst = []
mem_remain_fat_protein_lst = []
for i in range(len(main_df)):
    food_crb=main_df.iloc[i]['탄수화물(g)']
    food_cal=main_df.iloc[i]['열량(kcal)']
    food_protein=main_df.iloc[i]['단백질(g)']
    food_fat=main_df.iloc[i]['지방(g)']
    for j in range(len(mem_remain_cal_df)):
        remain_user_crb = mem_remain_cal_df.iloc[j]['remain_crb']
        remain_user_cal = mem_remain_cal_df.iloc[j]['remain_cal']
        remain_user_protein = mem_remain_cal_df.iloc[j]['remain_protein']
        remain_user_fat = mem_remain_cal_df.iloc[j]['remain_fat']
        
        if (remain_user_protein*0.65) <= food_protein <= (remain_user_protein*0.85) or (remain_user_fat*0.65)<= food_fat <= (remain_user_fat*0.85):
            food_main_lst.append(main_df.iloc[i])
            remain_crb = int(round(remain_user_crb - food_crb,1))
            remain_cal = int(round(remain_user_cal - food_cal,1))
            remain_protein = int(round(remain_user_protein - food_protein,1))
            remain_fat = int(round(remain_user_fat - food_fat,1))
            mem_remain_fat_protein_lst.append({'FOOD_MAIN_SEQ': main_df.iloc[i]['FOOD_SEQ'],'FOOD_SOUP_SEQ':mem_remain_cal_df.iloc[i]['FOOD_SOUP_SEQ'],'FOOD_RICE_SEQ':mem_remain_cal_df.iloc[j]['FOOD_RICE_SEQ'] ,
                                               'remain_crb':remain_crb,'remain_cal':remain_cal, 'remain_protein':remain_protein, 'remain_fat':remain_fat})

food_main_df = pd.DataFrame(food_main_lst)
mem_remain_fat_protein_df = pd.DataFrame(mem_remain_fat_protein_lst)

print(len(food_main_df))
print(len(mem_remain_fat_protein_df))

53970
53970


In [161]:
mem_remain_fat_protein_df

,FOOD_MAIN_SEQ,FOOD_SOUP_SEQ,FOOD_RICE_SEQ,remain_crb,remain_cal,remain_protein,remain_fat
0,2,22,122,-32,-233,-39,5
1,2,22,125,-28,-229,-42,5
2,2,22,146,-28,-215,-38,5
3,2,22,6982,-23,-163,-31,5
4,2,22,7021,-30,-190,-31,5
...,...,...,...,...,...,...,...
53965,7325,29,6974,-30,-198,0,-10
53966,7325,29,7000,-15,-115,1,-9
53967,7325,29,6986,-21,-256,0,-21
53968,7325,29,92,-36,-330,1,-23


In [162]:
# cal 마이너스인거 삭제하기
i = 0
while i < len(mem_remain_fat_protein_df):
    if mem_remain_fat_protein_df.iloc[i]['remain_cal']<0:
        mem_remain_fat_protein_df = mem_remain_fat_protein_df.drop(mem_remain_fat_protein_df.index[i])
    else:
        i += 1
print(len(mem_remain_fat_protein_df))

28116


In [164]:
# fat 마이너스 10 이상 인거 삭제하기
i = 0
while i < len(mem_remain_fat_protein_df):
    if mem_remain_fat_protein_df.iloc[i]['remain_fat']< (-10):
        mem_remain_fat_protein_df = mem_remain_fat_protein_df.drop(mem_remain_fat_protein_df.index[i])
    else:
        i += 1
print(len(mem_remain_fat_protein_df))

28116


In [165]:
# crb 마이너스 10 이상 인거 삭제하기
i = 0
while i < len(mem_remain_fat_protein_df):
    if mem_remain_fat_protein_df.iloc[i]['remain_crb']< (-10):
        mem_remain_fat_protein_df = mem_remain_fat_protein_df.drop(mem_remain_fat_protein_df.index[i])
    else:
        i += 1
print(len(mem_remain_fat_protein_df))

27173


In [166]:
# protein 마이너스 20 이상 인거 삭제하기
i = 0
while i < len(mem_remain_fat_protein_df):
    if mem_remain_fat_protein_df.iloc[i]['remain_protein']< (-20):
        mem_remain_fat_protein_df = mem_remain_fat_protein_df.drop(mem_remain_fat_protein_df.index[i])
    else:
        i += 1
print(len(mem_remain_fat_protein_df))

19115


In [167]:
mem_remain_fat_protein_df

,FOOD_MAIN_SEQ,FOOD_SOUP_SEQ,FOOD_RICE_SEQ,remain_crb,remain_cal,remain_protein,remain_fat
13,4,22,134,4,100,2,6
14,4,22,137,2,223,2,20
15,4,22,145,-4,124,2,13
16,4,22,6942,0,86,3,5
17,4,22,6944,9,161,5,9
...,...,...,...,...,...,...,...
53866,7325,29,7006,1,41,1,1
53867,7325,29,7021,-9,5,0,2
53885,7325,29,6956,-3,3,1,-1
53907,7325,29,146,-10,4,1,2


-그냥 반찬으로 부시기 (매무리)


In [151]:
sub_df = kor_jap_df[kor_jap_df['밥국반메면'].isin(['반'])]
sub_df

,FOOD_SEQ,음식명,레시피,한중일양,밥국반메면,중량,열량(kcal),탄수화물(g),단백질(g),지방(g)
4,5,더덕구이,https://www.youtube.com/embed/ir4Z2IyFIvM,한,반,100.0,184.00,31.10,3.10,5.20
43,44,도토리묵말이,https://www.youtube.com/embed/Ea4hrxvVbsU,한,반,1300.0,462.39,89.70,6.80,8.50
49,50,초당순두부,https://www.youtube.com/embed/FF0EPLm7cBQ,한,반,900.0,522.69,15.80,49.10,29.20
50,51,해파리냉채,https://www.youtube.com/embed/HpCKJKLkRG8,한,반,150.0,91.00,13.40,6.20,1.40
51,52,고들빼기,https://www.youtube.com/embed/tBvoXz6wM5A,한,반,50.0,55.53,10.93,1.58,0.61
...,...,...,...,...,...,...,...,...,...,...
7309,7310,감자튀김,https://www.youtube.com/embed/7d1uiasakhY,한,반,150.0,467.75,52.72,6.82,25.51
7311,7312,김말이튀김,https://www.youtube.com/embed/tabje0UKKyE,한,반,150.0,384.91,39.75,2.97,23.78
7312,7313,김튀김,https://www.youtube.com/embed/0ZgtTOHebcI,한,반,50.0,260.77,27.18,5.50,14.45
7314,7315,다시마튀각,https://www.youtube.com/embed/AWXFQfZwYds,한,반,50.0,230.86,23.66,4.22,13.26


In [168]:
food_sub_lst =[]
mem_final_lst = []
for i in range(len(sub_df)):
    food_crb=sub_df.iloc[i]['탄수화물(g)']
    food_cal=sub_df.iloc[i]['열량(kcal)']
    food_protein=sub_df.iloc[i]['단백질(g)']
    food_fat=sub_df.iloc[i]['지방(g)']
    for j in range(len(mem_remain_fat_protein_df)):
        remain_user_crb = mem_remain_fat_protein_df.iloc[j]['remain_crb']
        remain_user_cal = mem_remain_fat_protein_df.iloc[j]['remain_cal']
        remain_user_protein = mem_remain_fat_protein_df.iloc[j]['remain_protein']
        remain_user_fat = mem_remain_fat_protein_df.iloc[j]['remain_fat']
        
        if (remain_user_protein*0.70) <= food_protein <= remain_user_protein or (remain_user_fat*0.70)<= food_fat <= remain_user_fat or (remain_user_cal*0.7) <= food_cal <= remain_user_cal or (remain_user_crb*0.70) <= food_crb <= remain_user_crb:
            food_sub_lst.append(sub_df.iloc[i])
            remain_crb = int(round(remain_user_crb - food_crb,1))
            remain_cal = int(round(remain_user_cal - food_cal,1))
            remain_protein = int(round(remain_user_protein - food_protein,1))
            remain_fat = int(round(remain_user_fat - food_fat,1))
            mem_final_lst.append({'FOOD_RICE_SEQ':mem_remain_fat_protein_df.iloc[j]['FOOD_RICE_SEQ'] ,'FOOD_SOUP_SEQ':mem_remain_fat_protein_df.iloc[i]['FOOD_SOUP_SEQ'],'FOOD_MAIN_SEQ': mem_remain_fat_protein_df.iloc[i]['FOOD_MAIN_SEQ'],
                                  'FOOD_SUB_SEQ':sub_df.iloc[i]['FOOD_SEQ'],'remain_crb':remain_crb,'remain_cal':remain_cal, 'remain_protein':remain_protein, 'remain_fat':remain_fat})
food_sub_df = pd.DataFrame(food_sub_lst)
mem_final_df = pd.DataFrame(mem_final_lst)

print(len(food_sub_df))
print(len(mem_final_df))

1476806
1476806


In [171]:
# cal 마이너스인거 삭제하기
i = 0
while i < len(mem_final_df):
    if mem_final_df.iloc[i]['remain_cal']<0:
        mem_final_df = mem_final_df.drop(mem_final_df.index[i])
    else:
        i += 1
print(len(mem_final_df))

KeyboardInterrupt: 

In [ ]:
# fat 마이너스 10 이상 인거 삭제하기
i = 0
while i < len(mem_final_df):
    if mem_final_df.iloc[i]['remain_fat']<-10:
        mem_final_df = mem_final_df.drop(mem_final_df.index[i])
    else:
        i += 1
print(len(mem_final_df))

In [ ]:
# crb 마이너스 10 이상 인거 삭제하기
i = 0
while i < len(mem_final_df):
    if mem_final_df.iloc[i]['remain_crb']<-10:
        mem_final_df = mem_final_df.drop(mem_final_df.index[i])
    else:
        i += 1
print(len(mem_final_df))

In [ ]:
# protein 마이너스 20 이상 인거 삭제하기
i = 0
while i < len(mem_final_df):
    if mem_final_df.iloc[i]['remain_protein']<-20:
        mem_final_df = mem_final_df.drop(mem_final_df.index[i])
    else:
        i += 1
print(len(mem_final_df))